In [3]:
from openai import OpenAI
import requests
import json
import os
# 配置 API 密钥
STEP_API_KEY = os.getenv("STEPFUN_API_KEY") 
 
# 初始化 查询天气的接口参数，这里我使用的是高德查询天气接口，各位开发者可以按需自行选择供应商进行实现
get_weather_url = "https://restapi.amap.com/v3/weather/weatherInfo?parameters"
AMAP_API_KEY = os.getenv("AMAP_API_KEY")
COMPLETION_MODEL= "step-1-8k"
 
# 初始化 阶跃星辰 Client
client = OpenAI(base_url="https://api.stepfun.com/v1", api_key=STEP_API_KEY)
 

 
# 初始化查询天气的函数
def get_weather(city):
    params = {
        "city": city,
        "key": AMAP_API_KEY,
        "extensions": "base",  # 获取实时天气
    }
    response = requests.get(url=get_weather_url, params=params)  # 实时天气
    if response.status_code == 200:
        data = response.json()
        print(data)
        weather = data.get ('lives')[0].get ('weather')  # 当前天气现象
        temperature = data ['lives'][0].get ('temperature')  # 当前温度
        print ( f"{city} 现在的天气是 {weather} 温度是 {temperature}°C.")
    else:
        print("对不起，我无法获取这个地点的天气。")
 
 
# 构建 get_weather 函数说明
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "查询天气",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "城市名称（必须为汉字）"}
                },
                "required": ["city"],
            },
        },
    }
]
 
 
# 调用补全接口进行补全
def query_weather(prompt):
    completion = client.chat.completions.create(
        model=COMPLETION_MODEL,
        messages=[{"role": "user", "content": prompt}],
        ## 自动选择是否调用外部函数
        tool_choice="auto",
        tools=tools,
    )
    # print (completion.choices [0].message.content.strip ())
    print(completion)
    # 判断是否命中查询天气 Function
    if completion.choices[0].message.content.strip() == "":
        if completion.choices[0].message.tool_calls[0].function.name == "get_weather":
            # 获取参数
            city_name = json.loads(
                completion.choices[0].message.tool_calls[0].function.arguments
            )
            # 调用 Function
            get_weather(city_name["city"])
        else:
            print("未命中查询天气 Tool")
    else:
        print("抱歉出错了")
        # print (completion.choices [0].message.content.strip ())
 
 
# 查询天气
query_weather("上海实时天气")
 

ChatCompletion(id='fc0989175ae0569eb704f90c1f5e2450.326acc771346851ac8184295956a6e64', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_NCcUwVr1S3SsdIva8hu7eQ', function=Function(arguments='{"city": "上海"}', name='get_weather'), type='function', index=0)]))], created=1763632425, model='step-1-8k', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=92, total_tokens=108, completion_tokens_details=None, prompt_tokens_details=None))
{'status': '0', 'info': 'INVALID_USER_KEY', 'infocode': '10001'}


TypeError: 'NoneType' object is not subscriptable